<a href="https://colab.research.google.com/github/hmaldonado12/7506R-1C2024-GRUPO6/blob/main/tp2/TP2_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports y carga de archivos

In [1]:
# ----- PARAMETROS DEL OUTPUT -----
# MAX_ROWS: cantidad limite a mostrar, si se excede se resume (...)
# MIN_ROWS: cantidad de filas a mostrar pasado el corte
# MAX_COLWIDTH: ancho de columnas por cada output, si se excede se resume(...)

# pd.reset_option("display.max_rows")
# pd.options.display.max_rows
# pd.options.display.min_rows = 100000
# pd.options.display.max_colwidth = 1000

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import nltk
import re

In [4]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
train_df.head()

,ID,review_es,sentimiento
0,0,Uno de los otros críticos ha mencionado que de...,positivo
1,1,Una pequeña pequeña producción.La técnica de f...,positivo
2,2,Pensé que esta era una manera maravillosa de p...,positivo
3,3,"Básicamente, hay una familia donde un niño peq...",negativo
4,4,"El ""amor en el tiempo"" de Petter Mattei es una...",positivo


In [6]:
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv')
test_df.head()

,ID,review_es
0,60000,La mayor virtud de esta película es su existen...
1,60001,"No soy un experto cinéfilo, pero pocas veces m..."
2,60002,Si no eres un incondicional del humor estilo T...
3,60003,"No sé qué está pasando, si la gente se deja ll..."
4,60004,"Pero cuando amanece,y me quedo solo,siento en ..."


# Preprocesamiento de datos

Para el preprocesamiento de los datos hay que limpiar y preprocesar los textos para que los modelos puedan manejarlos adecuadamente.

In [7]:
# DEBUG: almaceno cada una de las palabras
# global_tokens = []

# Creo copias del dataset para procesar
token_train_df = train_df.copy()
token_test_df = test_df.copy()

def preprocess_text(text):
    # Llevo texto a minúsculas
    text = text.lower()

    # Expresión regular para eliminar caracteres '¿', '¡', y '`'
    text = re.sub(r'[¡¿\`\']', ' ', text)

    # Expresión regular para separar palabras con puntos intermedios
    text = re.sub(r'([\wáéíóúü]+)\.+([\wáéíóúü]+)', r'\1 \2', text)

    # Tokenizacion
    tokens = word_tokenize(text, language='spanish')

    # Elimino signos de puntuación
    tokens = [word for word in tokens if word not in string.punctuation]

    # Elimino stopwords
    stop_words = set(stopwords.words('spanish'))
    tokens = [word for word in tokens if word not in stop_words]

    # global_tokens.extend(tokens)
    return " ".join(tokens)

token_train_df['review_es'] = train_df['review_es'].apply(preprocess_text)
token_test_df['review_es'] = test_df['review_es'].apply(preprocess_text)

In [8]:
# len(global_tokens)

In [9]:
# Cuento cantidad de apariciones de palabras
# token_df = pd.DataFrame(global_tokens, columns=["Palabra"])
# token_df['Palabra'].value_counts()

In [10]:
# DEBUG: para buscar reseñas específicas
# token_train_df.loc[(token_train_df['review_es'].str.contains('``')), 'review_es']

In [11]:
token_train_df.head()

,ID,review_es,sentimiento
0,0,críticos mencionado después ver solo 1 oz epis...,positivo
1,1,pequeña pequeña producción técnica filmación i...,positivo
2,2,pensé manera maravillosa pasar tiempo fin sema...,positivo
3,3,básicamente familia niño pequeño jake piensa z...,negativo
4,4,`` amor tiempo '' petter mattei película visua...,positivo


In [12]:
token_test_df.head()

,ID,review_es
0,60000,mayor virtud película existencia hecho podamos...
1,60001,experto cinéfilo pocas veces tan juego sala ci...
2,60002,si incondicional humor estilo tele 5 si termin...
3,60003,sé pasando si gente deja llevar modas si verda...
4,60004,amanece quedo solo siento fondoun mar vacío se...


Feature Enconding:

Uso Bag of Words (BoW) para convertir el texto en vectores numéricos que puedan ser utilizados por los modelos de machine learning.

In [13]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(token_train_df['review_es'])
y_train = token_train_df['sentimiento']

In [14]:
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

//////////////////////////////////////////////////////////////////////

# XGBoost

In [15]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [16]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'eta': 0.1,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'verbosity': 0
}

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_split)
y_val_encoded = label_encoder.transform(y_val_split)

dtrain = xgb.DMatrix(X_train_split, label=y_train_encoded)
dval = xgb.DMatrix(X_val_split, label=y_val_encoded)
model = xgb.train(params, dtrain, num_boost_round=1000, evals=[(dval, 'validation')], early_stopping_rounds=10, verbose_eval=False)


Evaluación del modelo:

In [17]:
y_pred = model.predict(dval)
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]

Metricas:

In [18]:
y_val_pred_labels = label_encoder.inverse_transform(y_pred_binary)
y_val_true_labels = label_encoder.inverse_transform(y_val_encoded)

# Imprimir métricas de evaluación
print("Accuracy:", accuracy_score(y_val_true_labels, y_val_pred_labels))
print("Precision:", precision_score(y_val_true_labels, y_val_pred_labels, pos_label='positivo'))
print("Recall:", recall_score(y_val_true_labels, y_val_pred_labels, pos_label='positivo'))
print("F1 Score:", f1_score(y_val_true_labels, y_val_pred_labels, pos_label='positivo'))

print("Classification Report:\n", classification_report(y_val_true_labels, y_val_pred_labels))


Accuracy: 0.8694
Precision: 0.860258637328701
Recall: 0.8845008930343322
F1 Score: 0.872211350293542
Classification Report:
               precision    recall  f1-score   support

    negativo       0.88      0.85      0.87      4961
    positivo       0.86      0.88      0.87      5039

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



Realizar predicciones en el conjunto de prueba (test.csv)

In [19]:
X_test = vectorizer.transform(token_test_df['review_es'])
dtest = xgb.DMatrix(X_test)
test_predictions = model.predict(dtest)

Se decodifica las predicciones a etiquetas originales

In [20]:
test_predictions_labels = label_encoder.inverse_transform([1 if pred > 0.5 else 0 for pred in test_predictions])

Ahora se prepara el archivo csv para subrilo a kaggle

#Exportar CSV para Submission

In [21]:
# Creo nuevo DF con ID de reseña y predicción de sentimiento
output_df = pd.DataFrame({'ID': test_df['ID'], 'sentimiento': test_predictions_labels})
output_df

,ID,sentimiento
0,60000,negativo
1,60001,positivo
2,60002,negativo
3,60003,negativo
4,60004,negativo
...,...,...
8594,68594,positivo
8595,68595,negativo
8596,68596,positivo
8597,68597,negativo


In [22]:
# Genero archivo CSV
output_df.to_csv('/content/drive/My Drive/Colab Notebooks/XGBoost.csv', index=False)